In [1]:
from elasticsearch import Elasticsearch
from pysolr import Solr
from pathlib import Path
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
from collections import defaultdict
from operator import itemgetter
from pprint import pprint
from time import time

In [2]:
embeddings_path = Path('/media/discoD/Mestrado/NoLeak/gait/features-all/stored-features/')
random_seed = 123

references = [
    '1a6ce47f-d707-46d6-a7d9-3580e5897ca1', 
    '377f9fdd-b334-4d4c-a1b8-033fa8f9d575', 
    'e97d8728-4f62-454b-b361-669c3b9e2bd0', 
    'c5487d96-26d0-48ba-b64a-02590d1a0241', 
    '9aa93eee-3980-49c5-bed0-89ada3c35859'
]

In [3]:
def reduce_single_embedding(gait_embeddings, dimension):
    embeddings = np.array([embedding['gait_embedding_15872'].ravel() for embedding in gait_embeddings.values()])
    start = time()
    pca = PCA(n_components=dimension, random_state=random_seed)
    pca_result = pca.fit_transform(embeddings)
    print('PCA done! Time elapsed: {} seconds'.format(time()-start))
    start = time()
    svd = TruncatedSVD(n_components=dimension, random_state=random_seed)
    svd_result = svd.fit_transform(embeddings)
    print('SVD done! Time elapsed: {} seconds'.format(time()-start))
    start = time()
    tsne_result = TSNE(n_components=3, random_state=random_seed).fit_transform(embeddings)
    print('t-SNE done! Time elapsed: {} seconds'.format(time()-start))
    start = time()
    pca_dim = PCA(n_components=dimension, random_state=random_seed)
    pca_result_dim = pca_dim.fit_transform(embeddings)
    pca_tsne_result = TSNE(n_components=3, random_state=random_seed).fit_transform(pca_result_dim)
    print('t-SNE with PCA done! Time elapsed: {} seconds'.format(time()-start))
    return pca_result, svd_result, tsne_result, pca_tsne_result

def get_gait_embeddings(embeddings_path, reduce=True):
    embeddings = defaultdict(dict)
    for npy in embeddings_path.iterdir():
        if npy.name.endswith('.npy'):
            embedding = np.load(npy, allow_pickle=True)[0][0]
            assert len(embedding) == 15872
            uuid = npy.name.split('_')[0]
            embeddings[uuid] = {
                'uuid': uuid,
                'gait_embedding_15872': np.array(embedding).reshape(-1, 1),
                'gait_embedding_15872_str': ','.join([str(value) for value in embedding])
            }
    if reduce:
        pca_result, svd_result, tsne_result, pca_tsne_result = reduce_single_embedding(embeddings, 128)
        for pca_embedding, svd_embedding, tsne_embedding, pca_tsne_embedding, embedding in zip(pca_result, svd_result, tsne_result, pca_tsne_result, embeddings.values()):
            embedding['gait_embedding_pca_128'] = pca_embedding
            embedding['gait_embedding_svd_128'] = svd_embedding
            embedding['gait_embedding_tsne_3'] = tsne_embedding
            embedding['gait_embedding_pca_tsne_3'] = pca_tsne_embedding
    return embeddings

def get_similarities(gait_embeddings, references, embedding_type):
    similarities = defaultdict(list)
    for reference in references:
        gait_embedding = gait_embeddings.get(reference).get(embedding_type)
        similarities[reference].extend([{'uuid': embedding.get('uuid'), 'similarity': 1. - cosine(gait_embedding, embedding.get(embedding_type))} for embedding in gait_embeddings.values()])    
        similarities[reference] = sorted(similarities[reference], key=itemgetter('similarity'), reverse=True)
    return similarities

def get_similarity(uuid, similarities):
    for idx, similarity in enumerate(similarities):
        if similarity['uuid'] == uuid:
            return idx, similarity['similarity']

def score_similarities(similarities_1, similarities_2, debug=False):
#     print(type(similarities_1), type(similarities_2))
    scores = defaultdict(list)
    for uuid in similarities_1.keys():
        if debug:
            print('Scoring similarities for %s' % uuid)
        similarities_from_1 = similarities_1[uuid]
        similarities_from_2 = similarities_2[uuid]
#         print(len(similarities_from_1), len(similarities_from_2))
        for idx_from_1, similarity_1 in enumerate(similarities_from_1):
            score_from_1 = similarity_1['similarity']
#             print(uuid, idx_from_1)
            idx_from_2, score_from_2 = get_similarity(similarity_1['uuid'], similarities_from_2)
            if debug:
                print('Difference for %s:\n\t%s place in 1, %s score in 1, %s place in 2, %s score in 2' % (similarity_1['uuid'], idx_from_1, score_from_1, idx_from_2, score_from_2))
            score = abs(idx_from_1 - idx_from_2) * abs(score_from_1 - score_from_2)
            scores[uuid].append(score)
            if debug:
                print('\tScore: %s' % score)
    for uuid in scores.keys():
        scores_for_uuid = sum(scores[uuid])
        print('Scores for UUID %s: %s' % (uuid, scores_for_uuid))
    return scores

In [4]:
gait_embeddings = get_gait_embeddings(embeddings_path)
gait_embeddings['1a6ce47f-d707-46d6-a7d9-3580e5897ca1']

PCA done! Time elapsed: 0.16373801231384277 seconds
SVD done! Time elapsed: 0.32593321800231934 seconds
t-SNE done! Time elapsed: 2.4047605991363525 seconds
t-SNE with PCA done! Time elapsed: 2.5145576000213623 seconds


{'uuid': '1a6ce47f-d707-46d6-a7d9-3580e5897ca1',
 'gait_embedding_15872': array([[ 0.2340001 ],
        [-0.0394039 ],
        [-0.48620033],
        ...,
        [-0.11354651],
        [-0.43962842],
        [-0.1524397 ]], dtype=float32),
 'gait_embedding_15872_str': '0.2340001,-0.039403897,-0.48620033,-0.3670836,0.7413249,0.59998536,0.22727469,0.004756694,1.4453024,-0.46902302,0.4613026,-0.5085187,-0.5472903,-0.11946329,1.0277431,-0.50992554,-0.62619054,0.0042330706,-0.4726368,0.30670696,-0.3159612,-1.3833513,0.054267284,0.039239332,-0.008795229,0.06819675,-0.710605,-0.12904806,-0.6110883,0.76718056,0.7607554,-0.014684535,-0.34462023,0.034491923,0.69862276,-0.59157443,0.09349946,-0.8212857,0.48395604,-0.29508778,-0.54577476,0.9784899,0.7044309,-0.43664303,-0.36177912,0.550393,-0.2732327,-0.8727016,0.650135,1.1887909,0.22622278,0.64964885,-0.56242245,-1.1497293,-0.14267571,-0.65046304,0.740019,0.147129,-0.40419224,-0.566735,-0.20878091,-1.236529,-0.54130846,0.8121055,0.2042813,-0.189

In [5]:
similarities_15872 = get_similarities(gait_embeddings, references, 'gait_embedding_15872')
for reference in similarities_15872.keys():
    print('Reference UUID: %s' % reference)
    pprint(similarities_15872[reference][0:15])

Reference UUID: 1a6ce47f-d707-46d6-a7d9-3580e5897ca1
[{'similarity': 1.0, 'uuid': '1bee19cf-597c-4256-b035-f7cc1b7a8e96'},
 {'similarity': 1.0, 'uuid': '88b9f259-daaf-4903-a51f-3621c8917019'},
 {'similarity': 1.0, 'uuid': '7fc38cea-89bb-476f-82ce-2538dee7ac44'},
 {'similarity': 1.0, 'uuid': '1a6ce47f-d707-46d6-a7d9-3580e5897ca1'},
 {'similarity': 1.0, 'uuid': 'eff8490d-c2ed-4006-bcd1-5512f8fa56ec'},
 {'similarity': 0.9956472516059875,
  'uuid': 'd6c88698-2901-4f4d-ad3d-470c05550061'},
 {'similarity': 0.9956472516059875,
  'uuid': '651dfad5-73ab-4058-86bd-b3abff2b8135'},
 {'similarity': 0.9956472516059875,
  'uuid': 'be7caf03-52d3-4004-8cc7-258928780e39'},
 {'similarity': 0.9956472516059875,
  'uuid': '377f9fdd-b334-4d4c-a1b8-033fa8f9d575'},
 {'similarity': 0.9956472516059875,
  'uuid': 'fd3d02ad-e508-4623-a89a-75540625fb1e'},
 {'similarity': 0.9931745529174805,
  'uuid': 'e0720837-3190-48f1-a8cb-bf98c356308e'},
 {'similarity': 0.9931745529174805,
  'uuid': 'a4239a90-083e-4e9c-8584-e2ee

In [6]:
similarities_pca = get_similarities(gait_embeddings, references, 'gait_embedding_pca_128')
for reference in similarities_pca.keys():
    print('Reference UUID: %s' % reference)
    pprint(similarities_pca[reference][0:15])

Reference UUID: 1a6ce47f-d707-46d6-a7d9-3580e5897ca1
[{'similarity': 1.0, 'uuid': '7fc38cea-89bb-476f-82ce-2538dee7ac44'},
 {'similarity': 1.0, 'uuid': '1a6ce47f-d707-46d6-a7d9-3580e5897ca1'},
 {'similarity': 1.0, 'uuid': 'eff8490d-c2ed-4006-bcd1-5512f8fa56ec'},
 {'similarity': 0.9999998807907104,
  'uuid': '1bee19cf-597c-4256-b035-f7cc1b7a8e96'},
 {'similarity': 0.9999998807907104,
  'uuid': '88b9f259-daaf-4903-a51f-3621c8917019'},
 {'similarity': 0.9668868184089661,
  'uuid': 'd6c88698-2901-4f4d-ad3d-470c05550061'},
 {'similarity': 0.9668868184089661,
  'uuid': 'fd3d02ad-e508-4623-a89a-75540625fb1e'},
 {'similarity': 0.9668866991996765,
  'uuid': '651dfad5-73ab-4058-86bd-b3abff2b8135'},
 {'similarity': 0.9668866991996765,
  'uuid': 'be7caf03-52d3-4004-8cc7-258928780e39'},
 {'similarity': 0.9668866991996765,
  'uuid': '377f9fdd-b334-4d4c-a1b8-033fa8f9d575'},
 {'similarity': 0.946415901184082,
  'uuid': 'e0720837-3190-48f1-a8cb-bf98c356308e'},
 {'similarity': 0.9464158415794373,
  'uui

In [7]:
similarities_svd = get_similarities(gait_embeddings, references, 'gait_embedding_svd_128')
for reference in similarities_svd.keys():
    print('Reference UUID: %s' % reference)
    pprint(similarities_svd[reference][0:15])

Reference UUID: 1a6ce47f-d707-46d6-a7d9-3580e5897ca1
[{'similarity': 1.0, 'uuid': '88b9f259-daaf-4903-a51f-3621c8917019'},
 {'similarity': 1.0, 'uuid': '7fc38cea-89bb-476f-82ce-2538dee7ac44'},
 {'similarity': 1.0, 'uuid': '1a6ce47f-d707-46d6-a7d9-3580e5897ca1'},
 {'similarity': 1.0, 'uuid': 'eff8490d-c2ed-4006-bcd1-5512f8fa56ec'},
 {'similarity': 0.9999999403953552,
  'uuid': '1bee19cf-597c-4256-b035-f7cc1b7a8e96'},
 {'similarity': 0.9956470727920532,
  'uuid': 'd6c88698-2901-4f4d-ad3d-470c05550061'},
 {'similarity': 0.9956470727920532,
  'uuid': '651dfad5-73ab-4058-86bd-b3abff2b8135'},
 {'similarity': 0.9956470727920532,
  'uuid': 'be7caf03-52d3-4004-8cc7-258928780e39'},
 {'similarity': 0.9956470727920532,
  'uuid': '377f9fdd-b334-4d4c-a1b8-033fa8f9d575'},
 {'similarity': 0.9956470727920532,
  'uuid': 'fd3d02ad-e508-4623-a89a-75540625fb1e'},
 {'similarity': 0.9931745529174805,
  'uuid': 'e0720837-3190-48f1-a8cb-bf98c356308e'},
 {'similarity': 0.9931745529174805,
  'uuid': 'a4239a90-08

In [8]:
similarities_tsne = get_similarities(gait_embeddings, references, 'gait_embedding_tsne_3')
for reference in similarities_tsne.keys():
    print('Reference UUID: %s' % reference)
    pprint(similarities_tsne[reference][0:15])

Reference UUID: 1a6ce47f-d707-46d6-a7d9-3580e5897ca1
[{'similarity': 1.0, 'uuid': '1a6ce47f-d707-46d6-a7d9-3580e5897ca1'},
 {'similarity': 0.9993359446525574,
  'uuid': '06ff673d-5592-4331-a9f4-06d4b63bd2ee'},
 {'similarity': 0.9704459309577942,
  'uuid': 'ebcf3363-0673-483e-9124-5421a44d08ba'},
 {'similarity': 0.9464455246925354,
  'uuid': 'f6327613-3f7e-4582-b0f1-b98c78f403bb'},
 {'similarity': 0.8908503651618958,
  'uuid': '6a6a9fe5-20ed-49bd-9f1f-328d24bc7b3c'},
 {'similarity': 0.887913167476654,
  'uuid': '0b1c9e9f-f166-43bc-88db-94507580f106'},
 {'similarity': 0.8847877383232117,
  'uuid': '025d6f7e-5605-4284-861f-77513b240de9'},
 {'similarity': 0.8829397559165955,
  'uuid': 'd6c88698-2901-4f4d-ad3d-470c05550061'},
 {'similarity': 0.8657585978507996,
  'uuid': '60fa40be-6ab9-4859-9eab-f63dfdf3c11e'},
 {'similarity': 0.847637414932251,
  'uuid': '0866bee2-55a1-4698-b48f-5209631a7a23'},
 {'similarity': 0.8160989880561829,
  'uuid': 'cea3bf3f-5ce4-40fc-9f49-b003b738f2b6'},
 {'simila

In [9]:
similarities_pca_tsne = get_similarities(gait_embeddings, references, 'gait_embedding_pca_tsne_3')
for reference in similarities_pca_tsne.keys():
    print('Reference UUID: %s' % reference)
    pprint(similarities_pca_tsne[reference][0:15])

Reference UUID: 1a6ce47f-d707-46d6-a7d9-3580e5897ca1
[{'similarity': 1.0, 'uuid': '1a6ce47f-d707-46d6-a7d9-3580e5897ca1'},
 {'similarity': 0.9897953867912292,
  'uuid': '377f9fdd-b334-4d4c-a1b8-033fa8f9d575'},
 {'similarity': 0.9897006154060364,
  'uuid': 'be7caf03-52d3-4004-8cc7-258928780e39'},
 {'similarity': 0.974960446357727,
  'uuid': '88b9f259-daaf-4903-a51f-3621c8917019'},
 {'similarity': 0.9536987543106079,
  'uuid': 'e0720837-3190-48f1-a8cb-bf98c356308e'},
 {'similarity': 0.9525445699691772,
  'uuid': '1bee19cf-597c-4256-b035-f7cc1b7a8e96'},
 {'similarity': 0.9295611381530762,
  'uuid': '0b1c9e9f-f166-43bc-88db-94507580f106'},
 {'similarity': 0.9270031452178955,
  'uuid': 'a4239a90-083e-4e9c-8584-e2eecd05a9f0'},
 {'similarity': 0.9226308465003967,
  'uuid': 'e97d8728-4f62-454b-b361-669c3b9e2bd0'},
 {'similarity': 0.9072030186653137,
  'uuid': 'eff8490d-c2ed-4006-bcd1-5512f8fa56ec'},
 {'similarity': 0.8921210169792175,
  'uuid': '8bb10f19-845b-404f-a161-bcc695f27685'},
 {'simil

In [10]:
scores = score_similarities(similarities_15872, similarities_pca)

Scores for UUID 1a6ce47f-d707-46d6-a7d9-3580e5897ca1: 1975.639418205712
Scores for UUID 377f9fdd-b334-4d4c-a1b8-033fa8f9d575: 1953.035034801811
Scores for UUID e97d8728-4f62-454b-b361-669c3b9e2bd0: 1994.0721286591142
Scores for UUID c5487d96-26d0-48ba-b64a-02590d1a0241: 1964.0021483190358
Scores for UUID 9aa93eee-3980-49c5-bed0-89ada3c35859: 3230.754492191598


In [23]:
scores = score_similarities(similarities_15872, similarities_svd, debug=True)

Scoring similarities for 1a6ce47f-d707-46d6-a7d9-3580e5897ca1
Difference for 1bee19cf-597c-4256-b035-f7cc1b7a8e96:
	0 place in 1, 1.0 score in 1, 4 place in 2, 0.9999999403953552 score in 2
	Score: 2.384185791015625e-07
Difference for 88b9f259-daaf-4903-a51f-3621c8917019:
	1 place in 1, 1.0 score in 1, 0 place in 2, 1.0 score in 2
	Score: 0.0
Difference for 7fc38cea-89bb-476f-82ce-2538dee7ac44:
	2 place in 1, 1.0 score in 1, 1 place in 2, 1.0 score in 2
	Score: 0.0
Difference for 1a6ce47f-d707-46d6-a7d9-3580e5897ca1:
	3 place in 1, 1.0 score in 1, 2 place in 2, 1.0 score in 2
	Score: 0.0
Difference for eff8490d-c2ed-4006-bcd1-5512f8fa56ec:
	4 place in 1, 1.0 score in 1, 3 place in 2, 1.0 score in 2
	Score: 0.0
Difference for d6c88698-2901-4f4d-ad3d-470c05550061:
	5 place in 1, 0.9956472516059875 score in 1, 5 place in 2, 0.9956470727920532 score in 2
	Score: 0.0
Difference for 651dfad5-73ab-4058-86bd-b3abff2b8135:
	6 place in 1, 0.9956472516059875 score in 1, 6 place in 2, 0.9956470727

In [12]:
scores = score_similarities(similarities_15872, similarities_tsne)

Scores for UUID 1a6ce47f-d707-46d6-a7d9-3580e5897ca1: 4570.656098809675
Scores for UUID 377f9fdd-b334-4d4c-a1b8-033fa8f9d575: 3312.1549082882702
Scores for UUID e97d8728-4f62-454b-b361-669c3b9e2bd0: 2711.8787514716387
Scores for UUID c5487d96-26d0-48ba-b64a-02590d1a0241: 2675.112919975072
Scores for UUID 9aa93eee-3980-49c5-bed0-89ada3c35859: 5200.050599764334


In [13]:
scores = score_similarities(similarities_15872, similarities_pca_tsne)

Scores for UUID 1a6ce47f-d707-46d6-a7d9-3580e5897ca1: 4264.310634195805
Scores for UUID 377f9fdd-b334-4d4c-a1b8-033fa8f9d575: 4063.379628781695
Scores for UUID e97d8728-4f62-454b-b361-669c3b9e2bd0: 4895.292540103197
Scores for UUID c5487d96-26d0-48ba-b64a-02590d1a0241: 4330.104524590075
Scores for UUID 9aa93eee-3980-49c5-bed0-89ada3c35859: 4585.490859043784


In [14]:
client = Elasticsearch('localhost', port=9200)

In [15]:
# client.index(index='gait', doc_type='gait_embedding', body={})

In [16]:
# def index_gait_embeddings(file_name, base_path = Path('/media/discoD/Mestrado/NoLeak/gait/')):
#     file_path = base_path / file_name
#     embedding = np.load(file_path, allow_pickle=True)[0][0]
#     assert len(embedding) == 15872
#     uuid = file_name.split('_')[0]
#     print(uuid)
#     solr_client.add([
#         {
#             'uuid': uuid,
#             'template_8192': ','.join([str(value) for value in embedding[:8192]])
#         }
#     ])

In [17]:
# for idx, gait_embedding in enumerate(gait_embeddings.values()):
#     if gait_embedding.get('gait_embedding_pca_128') is not None:
#         body={'uuid': gait_embedding.get('uuid'), 'gait_embedding_pca_128': gait_embedding.get('gait_embedding_pca_128').tolist()}
#         client.index(index='gait', body=body)
#         print('Finished %s index: %s' % (idx, gait_embedding.get('uuid')))

In [18]:
for gait_embedding in gait_embeddings.values():
    print(gait_embedding)
    break

{'uuid': '7bf3ae05-d8e2-492e-8df4-2c8aff789571', 'gait_embedding_15872': array([[-0.15594739],
       [ 0.36396274],
       [-1.0169462 ],
       ...,
       [-0.1335995 ],
       [-0.5374372 ],
       [-0.44980532]], dtype=float32), 'gait_embedding_15872_str': '-0.15594739,0.36396274,-1.0169462,-0.1902085,0.9363024,0.2946885,0.1602736,0.0644356,0.856489,-0.76329297,0.369143,0.024816472,0.30615553,-0.22786936,1.1026343,-0.4834647,-0.6174917,-0.34202513,-0.58427376,0.29367337,-0.27890885,-0.7900168,0.3511154,0.061086185,0.836087,0.16391256,0.16438456,-0.44346035,-0.21436079,0.14666358,0.16011481,-0.10765038,0.3370708,-0.33247453,0.81349736,-0.58701986,0.096486315,-0.6553107,0.021144044,-0.79322785,0.12350961,0.3018385,0.86100996,0.04958084,0.0069411136,0.9699756,-0.47568196,-0.58344394,0.53222483,0.52573824,0.6420884,0.702545,-0.13732672,-0.8992389,-0.30432633,0.35235664,0.8119454,0.70147085,0.14322804,-0.706484,0.084937505,-1.1516263,-0.49468905,0.39580393,-0.100036055,0.1479405,1.1258

In [19]:
len(gait_embeddings.values())

136

In [20]:
list(gait_embeddings.values())[-2]

{'uuid': '160480a6-69ef-4f1e-82bf-2ab4789963bd',
 'gait_embedding_15872': array([[-0.05426353],
        [-0.27246088],
        [-0.89993376],
        ...,
        [-0.1269353 ],
        [-0.6085899 ],
        [-0.15638079]], dtype=float32),
 'gait_embedding_15872_str': '-0.054263532,-0.27246088,-0.89993376,-0.07483395,0.39281294,0.52615905,0.37534755,0.053059872,0.93385434,-0.35633475,0.17938304,-0.55446965,-0.039834622,0.15739752,1.1712065,0.0056781294,-0.40824005,-0.51153576,-0.46979824,0.27769664,0.03903676,-0.8518969,-0.056644257,0.24654357,0.16527696,0.3954063,-0.0786609,0.2464236,0.062024534,0.6662254,0.6047084,-0.19498824,0.2219916,0.09081776,0.51617104,-0.78450817,0.4014762,-0.5795395,0.36642188,-0.53304017,-0.3039281,0.34564015,0.8399265,-0.37134796,0.13170037,0.8943437,-0.24296005,-0.6880633,0.41933885,0.80717427,0.29182464,0.7909074,-0.37151527,-0.94417447,0.08221512,-0.26997864,0.5537148,0.46344405,-0.01391501,-0.5948926,-0.080161236,-1.1243842,-0.6098695,0.25059876,0.07558

In [21]:
def search_by_es_similarity(reference_embedding):
    query = {
      "query": {
        "script_score": {
          "query" : {
            "match_all": {}
          },
          "script": {
            "source": "cosineSimilarity(params.query_vector, doc['gait_embedding_pca_128']) + 1.0", 
            "params": {
              "query_vector": reference_embedding
            }
          }
        }
      },
      "size": 136
    }
    return client.search(index='gait', body=query)

In [22]:
similarities_elastic = defaultdict(list)
for reference in references:
    print('Scores for %s' % reference)
    response = search_by_es_similarity(gait_embeddings[reference]['gait_embedding_pca_128'].tolist())
    for idx, hit in enumerate(response['hits']['hits']):
        uuid = hit['_source']['uuid']
        score = hit['_score'] - 1
        print('Score for UUID %s (%s): %s' % (uuid, idx, score))
        similarities_elastic[reference].extend([{'uuid': uuid, 'similarity': score}])    
    similarities_elastic[reference] = sorted(similarities_elastic[reference], key=itemgetter('similarity'), reverse=True)

Scores for 1a6ce47f-d707-46d6-a7d9-3580e5897ca1


ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x7fd3db84e390>: Failed to establish a new connection: [Errno 111] Connection refused) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7fd3db84e390>: Failed to establish a new connection: [Errno 111] Connection refused)

In [ ]:
similarities_pca

In [ ]:
similarities_elastic

In [ ]:
score_similarities(similarities_15872, similarities_pca)

In [ ]:
score_similarities(similarities_15872, similarities_elastic)